# Ransomware decision tree classifier

In [182]:
import numpy as np
import pandas as pd
import warnings
from IPython.display import display, HTML
from collections import Counter

# Suppress all warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_seq_items', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# read the dataset
dataSet = pd.read_csv('dataset/ransom_forest_class_ip.csv', encoding = 'unicode_escape')
dataSet = dataSet.iloc[:, 1:]

# dataSet = dataSet.drop(['Flow ID',' Timestamp'],axis=1)
# dataSet = pd.concat([dataSet[' Timestamp'],dataSet['Label'],dataSet['Flow ID']],axis=1)
dataSet.head()

,Source IP,Source Port,Flow IAT Min,Flow IAT Mean,Flow IAT Max,Flow Duration,Flow Packets/s,Fwd Packets/s,Init_Win_bytes_forward,Fwd IAT Min,Bwd Packets/s,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Total,Flow Bytes/s,Init_Win_bytes_backward,Destination Port,Flow IAT Std,Fwd IAT Std,Average Packet Size,Label
0,0,0.662169,-0.230795,-0.368331,-0.499224,-0.479053,-0.143198,-0.128846,1.377379,-0.223468,-0.105075,-0.349775,-0.422875,-0.393004,-0.068145,-0.193024,-0.3589,-0.406360,-0.335907,1.329492,Benign
1,0,0.662169,-0.230743,-0.369939,-0.501952,-0.485911,-0.035761,-0.009968,-0.704224,-0.223433,-0.109208,-0.352390,-0.425028,-0.398424,-0.042102,-0.233953,-0.3589,-0.408790,-0.338050,-0.545069,Benign
2,0,-0.238853,-0.230780,-0.369972,-0.501965,-0.485922,0.175303,0.221401,-0.707740,-0.223467,-0.109208,-0.352420,-0.425044,-0.398436,0.074325,-0.233953,-0.3589,-0.408790,-0.338050,-0.545069,Benign
3,0,0.903280,-0.224925,-0.364700,-0.499898,-0.484241,-0.143935,-0.129112,-0.705722,-0.223484,-0.107102,-0.352436,-0.425053,-0.398442,-0.101575,-0.188630,-0.3589,-0.408790,-0.338050,-0.505671,Benign
4,0,0.330493,-0.230790,-0.367863,-0.498489,-0.477793,-0.143578,-0.128975,1.377379,-0.223459,-0.106151,-0.349157,-0.420010,-0.391740,-0.094055,-0.193140,-0.3589,-0.405693,-0.333292,-0.044641,Benign


In [184]:
train_ratio = 0.2
n_samples = dataSet.shape[0]
indices = np.random.permutation(n_samples)

# Train/test split
train_size = int(train_ratio * n_samples)
train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_data = dataSet.iloc[train_indices]
test_data = dataSet.iloc[test_indices]

In [185]:
def split_Xy(df):
    X = df.drop('Label', axis=1)
    y = df['Label']
    return X, y

In [186]:
X_train, y_train = split_Xy(train_data)
X_test, y_test = split_Xy(test_data)

## Tree classes

**Node class**

In [187]:
class Node:
  def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None


### Decision tree

In [188]:
class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_features = n_features
    self.root = None

  def fit(self, X, y):
    self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
    self.root = self._grow_tree(X, y)

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_feats = X.shape
    n_labels = len(np.unique(y))

    #check the stopping criteria
    if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
      leaf_value= self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

    #find the best split
    best_feature, best_thresh = self._best_split(X, y, feat_idxs)
    if best_feature == None and best_thresh == None :
      return Node(value=self._most_common_label(y))

    #create child nodes
    left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
    return Node(best_feature, best_thresh, left, right)

  def _best_split(self, X, y, feat_idxs):
    best_gain = -1
    split_idx, split_thresh = None, None

    for feat_idx in feat_idxs:
      X_column = X[:, feat_idx]
      thresholds = np.unique(X_column) #list of unique values
      if len(thresholds) > 20:
        # Limit to 20 best thresholds (quantile-based)
        # thresholds = np.percentile(X_column, np.linspace(0, 100, 20))
        thresholds = np.linspace(np.min(X_column), np.max(X_column), 40)

      for thr in thresholds:
        #calculate information gain
        gain = self._information_gain(y, X_column, thr)

        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_thresh = thr

    return split_idx, split_thresh

  def _information_gain(self, y, X_column, threshold):
    #parent entropy
    parent_entropy = self._entropy(y)

    #create children
    left_idxs, right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return -2

    # calculate the weighted avg entropy of children

    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

    #calculate the information gain
    information_gain = parent_entropy - child_entropy
    return information_gain

  def _split(self, X_column, split_thresh):
    left_idxs = np.argwhere(X_column <= split_thresh).flatten()
    right_idxs = np.argwhere(X_column > split_thresh).flatten()
    return left_idxs, right_idxs

  def _entropy(self, y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log(p) for p in ps if p > 0])

  def _most_common_label(self, y):
    counter = Counter(y)
    value = counter.most_common(1)[0][0]
    return value

  def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)



### Random forest

In [189]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.n_features=n_feature
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                            min_samples_split=self.min_samples_split,
                            n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, int(0.75 * n_samples), replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions

## Test results

In [190]:
label_mapping = {label: idx for idx, label in enumerate(y_train.unique())}
y_train = y_train.map(label_mapping)
y_test = y_test.map(label_mapping)

In [191]:
sampled_indices = X_train.sample(frac=0.1, random_state=42).index
x_sampled = X_train.loc[sampled_indices].to_numpy()
y_sampled = y_train.loc[sampled_indices].to_numpy()

In [192]:
dt = DecisionTree(min_samples_split = 5, max_depth = 17)
dt.fit(x_sampled,y_sampled)
y_pred = dt.predict(X_test.values)

In [193]:
accuracy = np.mean(y_test == y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 34.63%


In [194]:
clf = RandomForest(n_trees=20, max_depth=20, min_samples_split=3, n_feature=5)
clf.fit(x_sampled, y_sampled)
y_pred = clf.predict(X_test.values)
print(y_pred)
accuracy = np.mean(y_test == y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

[0 2 4 ... 7 6 2]
Accuracy: 36.03%
